# Overview 

### This notebook extracts data from files that contain air quality and real estate data, and then transforms and loads the data into a SQL database. 
* File containing air quality data: aqi_yearly_1980_to_2021.csv 
* File containing real estate data: RDC_Inventory_Core_Metrics_County.csv
* File containing US state abbreviations: state_abbreviation.csv



In [ ]:
# load pandas and object relational mapper modules
import pandas as pd
from sqlalchemy import create_engine

## Extract CSVs into DataFrames

In [ ]:
# Extract air quality data from .csv file into dataframe
air_quality_file = "Resources/aqi_yearly_1980_to_2021.csv"
air_quality_file_df = pd.read_csv(air_quality_file)
air_quality_file_df.head()

In [ ]:
# Extract real estate data from .csv file into dataframe
rdc_listings_file = "Resources/RDC_Inventory_Core_Metrics_County.csv"
rdc_listings_file_df = pd.read_csv(rdc_listings_file)
rdc_listings_file_df.head()

### Transform Air Quality DataFrame

In [ ]:
# Select the columns that will be transformed from the air_quality_file_df dataframe
air_quality_file_cols = ["State", "County", "Year","Median AQI"]

# load the data from the selected columns into a new dataframe
air_quality_file_transformed_df= air_quality_file_df[air_quality_file_cols].copy()
air_quality_file_transformed_df.head()

In [ ]:
# Create a filtered dataframe that only contain the data for the year 2021
air_quality_filtered_df = air_quality_file_transformed_df.loc[air_quality_file_transformed_df["Year"] == 2021, :]
air_quality_filtered_df.head()

In [ ]:
# rename the columnsto lower cases
air_quality_df = air_quality_filtered_df.rename(columns={"State": "state",
                                                         "County":"county",
                                                         "Year": "year",
                                                         "Median AQI": "median_aqi"})
air_quality_df.head()

In [ ]:
# name the index as "id", this is the transformed air quality dataframe that is ready to load into database
air_quality_df.index.name = 'id'
air_quality_df.head()

### Transform Real Estate Listings DataFrame

In [ ]:
# Select the columns that will be transformed from the rdc_listings_file_df dataframe
rdc_listings_file_cols = ["month_date_yyyymm", "county_name", "total_listing_count", "average_listing_price", 
                          "median_listing_price", "active_listing_count", "median_days_on_market"]
listings_file_transformed_df= rdc_listings_file_df[rdc_listings_file_cols].copy()
listings_file_transformed_df.head()

In [ ]:
# split the column "county_name" to two columns: "County" and "State"
listings_file_transformed_df[['County', 'State']] = listings_file_transformed_df['county_name'].str.split(', ', 1, expand=True)
listings_file_transformed_df.head()

In [ ]:
# re-orginize the dataframe by selecting columns and assign to a new dataframe
df = listings_file_transformed_df[["month_date_yyyymm", "County", "State","total_listing_count", "average_listing_price", 
                                   "median_listing_price", "active_listing_count", "median_days_on_market"]]
df.head()

In [ ]:
# change the strings in the "State" column to upper cases
df["State"] = df["State"].str.upper()

# change the strings in the "County" column to title (upper case for the first letter in each word)
df["County"] = df["County"].str.title()
df.head()

Since the "State" column in the air quality dataframe (air_quality_df) is shown as full name and the "State" column in the real estate dataframe (df) is shown as state code, we decided to load in a .csv file that contain the state abbreviation information. 

In [ ]:
#Bring in state name - state abbreviation conversion table
state_abbreviation_file = "Resources/state_abbreviation.csv"
state_abbreviation_file_df = pd.read_csv(state_abbreviation_file)
state_abbreviation_file_df.head()

In [ ]:
# merge the real estate dataframe (df) with state abbreviation dataframe
merge_df = pd.merge(df, state_abbreviation_file_df, left_on="State", right_on="Code")

# change the data type for "average_listing_price" from "float64" to "int64", and assign to a new column
merge_df['Average_price']=merge_df['average_listing_price'].astype('int64')
merge_df.head()

In [ ]:
# Select the columns that will be used from the merged dataframe
listings_df = merge_df[["County", "State_x","State_y","total_listing_count", "Average_price", 
                        "median_listing_price", "active_listing_count", "median_days_on_market"]]

# rename the columns to lower cases
listings_df = listings_df.rename(columns={"month_date_yyyymm": "current_month",
                                          "County":"county",
                                          "State_x":"state_initial",
                                          "State_y": "state",
                                          "Average_price": "average_price"})
listings_df.head()

In [ ]:
# name the index as "id", this is the transformed real estate listing dataframe that is ready to load into database
listings_df.index.name = 'id'
listings_df.head()

## Create database connection
Before running the following code, a database needs to be initiallized on local server. Here are the steps to create an empty database in pgAdmin:
1. Create a database called "realestate_db".
2. Create two tables "listings" and "air_quality" in the realestate_db database using the following query code:

        CREATE TABLE listings (
          id INT PRIMARY KEY,
          county TEXT,
          state_initial TEXT,
          state TEXT,
          total_listing_count INT,
          average_price INT,
          median_listing_price INT,
          active_listing_count INT,
          median_days_on_market INT
        );



        CREATE TABLE air_quality (
          id INT PRIMARY KEY,
          state TEXT,
          county TEXT,
          year INT,
          median_aqi INT
        );


In [ ]:
# # Create our database engine
connection_string = "postgres:bootcamp@localhost:5432/realestate_db"
engine = create_engine(f'postgresql://{connection_string}')

In [ ]:
# Confirm the tables in the "realestate_db" database
engine.table_names()

## Load DataFrames into database

In [ ]:
# Use pandas to load csv converted air quality DataFrame into database
air_quality_df.to_sql(name='air_quality', con=engine, if_exists='append', index=True)

In [ ]:
# Use pandas to load csv converted listing DataFrame into database
listings_df.to_sql(name='listings', con=engine, if_exists='append', index=True)

In [ ]:
# Confirm data has been added by querying the "listings" table
pd.read_sql_query('select * from listings', con=engine).head()

In [ ]:
# Confirm data has been added by querying the "air_quality" table
pd.read_sql_query('select * from air_quality', con=engine).head()